# 预处理效果测试
主要是检验boundingbox的生成情况

In [ ]:
import os
os.chdir('..')

import cv2
import csv
import numpy as np

from PIL import Image

from preprocess.datagen import DataGenerator
from config import Config

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
def check_image(name):
#     name = train_table[i]
    data = data_dict[name]
    bound = data['bound']
    points = data['points']
    print(name)

    width = bound[2] - bound[0] + 1
    height = bound[3] - bound[1] + 1
    print(width, height)

    image = cv2.imread(os.path.join('data/train/blouse/Images', name))
    raw = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    for i in range(points.shape[0]):
        cv2.circle(raw, (points[i][0], points[i][1]), 7, (0, 255, 0), -1)
    plt.figure(1)
    plt.imshow(raw)

    points = points - np.array(bound[0:2])
    print(points)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image[bound[1]:(bound[3]+1), bound[0]:(bound[2]+1)]
    for i in range(points.shape[0]):
        cv2.circle(image, (points[i][0], points[i][1]), 3, (255, 0, 0), -1)
    plt.figure(2)
    plt.imshow(image)

## 检查关键点是否丢失
遍历一个类的bbox，统计关键点在bbox内的比例

In [ ]:
i = 0
out_of_range = 0
resize_rate = 0

error = False
while i < len(train_table) - 1:
    name = train_table[i]
    data = data_dict[name]
    bound = data['bound']
    points = data['points']
    weight = data['weight']
    
    im = Image.open(os.path.join('data/train/blouse/Images', name))
    orig_width, orig_height = im.size
    
    bound[0] = max(0, bound[0])
    bound[1] = max(0, bound[1])
    bound[2] = min(orig_width, bound[2])
    bound[3] = min(orig_height, bound[3])
    width = bound[2] - bound[0] + 1
    height = bound[3] - bound[1] + 1

    new_points = points - np.array(bound[0:2])
    data['new_points'] = new_points

    for j in range(new_points.shape[0]):
        if weight[j] != -1:
            if (new_points[j][0] > width) or (
                new_points[j][0] < 0) or (
                new_points[j][1] > height) or (
                new_points[j][1] < 0):
                
                out_of_range += 1
                check_image(name)
                error = True
                break
    
#     if error:
#         break
        
    i += 1
    
print('Invalid rate:', out_of_range / i, '%')

## 可视化bbox和关键点
按bbox切割图像，并对比关键点位置

In [ ]:
cfg = Config()
dataset = DataGenerator(cfg)
dataset.generate_set(train = True)

In [ ]:
train_generator = dataset.generator(cfg.img_size, cfg.hm_size, 1, cfg.nStacks, 'train')

In [ ]:
img, hm0, hm1, hm2, weight = next(train_generator)
image = img[0]
hm = hm2[0]
weight = weight[0] 

raw_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
raw_image = cv2.resize(raw_image, (256, 256), interpolation=cv2.INTER_LINEAR)

# visualize ground truth
gt_hm_visual = np.sum(hm, 2) * 255
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = raw_image + gt_hm_visual.astype(np.int32)
plt.imshow(gt_visual)
plt.show()

In [ ]:
weight = [0, 0, 1]
text = [1, 2, 3]
[text[i] for i in range(len(text)) if weight[i] == 1]

## Fallback
relax bbox of outwear category to the whole image size

In [ ]:
with open('data/train/outwear/points.txt') as infile:
    with open('data/train/outwear/points_row_bound.txt', 'w') as outfile:
        spam_reader = csv.reader(infile, delimiter=' ')
        spam_writer = csv.writer(outfile, delimiter=' ')
        for row in spam_reader:
            name = row[0]
            points = row[5:]
            im = Image.open(os.path.join('data/train/outwear/Images', name))
            orig_width, orig_height = im.size
            bound = [0, 0, orig_width, orig_height]
            spam_writer.writerow([name] + bound + points)